In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.utils import class_weight
import numpy as np
from sklearn.metrics import classification_report

# Define the L2 regularization values to test
l2_values = [0.00001, 0.0001, 0.001, 0.01, 0.1]

# Data generators
img_size = (224, 224)
batch_size = 32
train_dir = 'D:\\Splitted Curated X-Ray Dataset\\train'
val_dir = 'D:\\Splitted Curated X-Ray Dataset\\val'
test_dir = 'D:\\Splitted Curated X-Ray Dataset\\test'

train_datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.2, horizontal_flip=True, fill_mode='constant', cval=0, rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=1, class_mode='categorical', shuffle=False)

# Compute class weights
class_counts = np.unique(train_generator.classes, return_counts=True)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights_dict = {class_label: weight for class_label, weight in zip(class_counts[0], class_weights)}

# Define the initial learning rate and the learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

# Set up early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Store the results
results = []

results = []
histories = {}

for l2_value in l2_values:
    # Load the ResNet50V2 model with pretrained weights (excluding the top dense layers)
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze the convolutional layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Create a new model on top
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L2(l2_value)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    
    # Define the optimizer with the learning rate schedule
    opt = optimizers.Adam(learning_rate=lr_schedule)
    
    # Compile the model
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=50, validation_data=val_generator, validation_steps=val_generator.samples // batch_size, class_weight=class_weights_dict)
    
    # Store the history
    histories[l2_value] = history
    
    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(val_generator, steps=len(val_generator))
    results.append((l2_value, val_loss, val_accuracy))
    
    print(f'L2 Value: {l2_value}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
    
    # Generate and print the classification report
    y_pred = model.predict(test_generator)
    y_true = test_generator.classes
    y_pred_labels = tf.argmax(y_pred, axis=1)
    class_labels = list(test_generator.class_indices.keys())
    print(f"\nClassification Report for L2 Value {l2_value}:")
    print(classification_report(y_true, y_pred_labels, target_names=class_labels, digits=4))

# Determine the best L2 value
best_l2_value = min(results, key=lambda x: x[1])[0]
print(f'Best L2 Value: {best_l2_value}')

# Train the final model with the best L2 value
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L2(best_l2_value)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

opt = optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=50, validation_data=val_generator, validation_steps=val_generator.samples // batch_size, class_weight=class_weights_dict)

# Evaluate the final model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

y_pred = model.predict(test_generator)
y_true = test_generator.classes
y_pred_labels = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report for Best L2 Value:")
print(classification_report(y_true, y_pred_labels, target_names=class_labels, digits=4))

# Save the final model
model.save('D:/UNAIR/SKRIPSI/Seminar Proposal/FIX/Kodingan/Saved Model/resnet50_adam_100_best_l2.keras')
print("Model saved successfully.")

for l2_value, history in histories.items():
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model Accuracy for L2 Value {l2_value}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss for L2 Value {l2_value}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()

Found 7364 images belonging to 4 classes.
Found 920 images belonging to 4 classes.
Found 924 images belonging to 4 classes.
Epoch 1/50
230/230 [==============================] - 169s 722ms/step - loss: 2.8362 - accuracy: 0.6446 - val_loss: 0.4967 - val_accuracy: 0.7991
Epoch 2/50
230/230 [==============================] - 163s 707ms/step - loss: 0.6980 - accuracy: 0.7047 - val_loss: 0.4969 - val_accuracy: 0.7757
Epoch 3/50
230/230 [==============================] - 161s 699ms/step - loss: 0.6611 - accuracy: 0.7155 - val_loss: 0.7066 - val_accuracy: 0.7679
Epoch 4/50
230/230 [==============================] - 161s 699ms/step - loss: 0.6403 - accuracy: 0.7114 - val_loss: 0.4538 - val_accuracy: 0.8092
Epoch 5/50
230/230 [==============================] - 161s 698ms/step - loss: 0.6380 - accuracy: 0.7154 - val_loss: 0.4578 - val_accuracy: 0.8170
Epoch 6/50
230/230 [==============================] - 162s 703ms/step - loss: 0.6356 - accuracy: 0.7265 - val_loss: 0.4353 - val_accuracy: 0.8114


KeyboardInterrupt: 